In [1]:
from vector_db import generate_embedding

# Generate and print the embedding for a test document
test_summary = "This is a test summary."
embedding = generate_embedding(test_summary)
print(embedding)

d:\Github\AI-Summary-Paper-Tool\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.0442616268992424, 0.038675807416439056, 0.0017315897857770324, 0.08350192755460739, 0.041021861135959625, -0.012498625554144382, -0.04947070777416229, 0.055504146963357925, 0.006562558002769947, 0.021547680720686913, 0.055765602737665176, -0.017521951347589493, -0.01358579657971859, 0.03202340006828308, -0.03873199224472046, -0.06782695651054382, 0.024036886170506477, -0.06685757637023926, -0.02506704069674015, 0.039112482219934464, 0.07562921196222305, 0.048091620206832886, 0.03195182979106903, 0.033250585198402405, 0.017094243317842484, 0.023204810917377472, -0.054932788014411926, 0.0656362846493721, -0.03289870172739029, -0.02269596979022026, -0.014540634118020535, 0.08610251545906067, 0.07885034382343292, 0.055730190128088, -0.009773613885045052, 0.0006948455120436847, 0.03793064504861832, 0.07355150580406189, 0.0009711651364341378, 0.02588220313191414, 0.025184057652950287, -0.0683157742023468, 0.05492625758051872, 0.07081159204244614, -0.03436018526554108, -0.01760591194033622

In [5]:
import chromadb
import json
import numpy as np

# Initialize Chroma client
client = chromadb.PersistentClient(path="chroma_db")  # Replace with your Chroma DB path

# Get or create the collection
collection = client.get_or_create_collection(name="paper_summaries")

def inspect_embeddings():
    """
    Retrieve and inspect all embeddings, documents, and metadata from the Chroma database.
    """
    try:
        # Retrieve all data from the collection
        results = collection.get(include=["embeddings", "documents", "metadatas"])
        
        # Extract components
        ids = results["ids"]
        documents = results["documents"]
        metadatas = results["metadatas"]
        embeddings = results["embeddings"]

        # Convert embeddings (NumPy arrays) to Python lists
        embeddings = [emb.tolist() if isinstance(emb, np.ndarray) else emb for emb in embeddings]

        # Print the retrieved data
        print("=== Retrieved Data ===")
        for i, (id_, doc, meta, emb) in enumerate(zip(ids, documents, metadatas, embeddings)):
            print(f"\nEntry {i + 1}:")
            print(f"ID: {id_}")
            print(f"Document: {doc}")
            print(f"Metadata: {meta}")
            print(f"Embedding (first 5 dimensions): {emb[:5]}...")  # Show only the first 5 dimensions for brevity
        
        # Save the data to a JSON file
        output_data = {
            "ids": ids,
            "documents": documents,
            "metadatas": metadatas,
            "embeddings": embeddings  # Embeddings are now Python lists
        }
        with open("chroma_inspection.json", "w") as f:
            json.dump(output_data, f, indent=4)
        
        print("\nData has been saved to 'chroma_inspection.json'.")

    except Exception as e:
        print(f"Error inspecting embeddings: {str(e)}")

# Run the inspection
if __name__ == "__main__":
    inspect_embeddings()

=== Retrieved Data ===

Entry 1:
ID: GR1_AI-Translation-Project-1
Document: Dưới đây là bản tóm tắt bài báo:

## Research Paper Summary

### Title: Mô Hình Phiên Dịch Ngôn Ngữ

1.  **Research Problem:**
    *   Mục tiêu chính của nghiên cứu là phát triển một ứng dụng dịch ngôn ngữ từ tiếng Anh sang tiếng Việt dựa trên trí tuệ nhân tạo (AI) để cung cấp bản dịch chất lượng cao, chính xác và dễ sử dụng.
    *   Bài báo giải quyết vấn đề thiếu hụt các tập dữ liệu song song chất lượng cao và đủ lớn cho tiếng Việt, một ngôn ngữ tài nguyên thấp trong lĩnh vực dịch máy.

2.  **Hypothesis/Research Questions:**
    *   Bài báo không đặt ra các giả thuyết cụ thể. Thay vào đó, nó tập trung vào việc xây dựng và đánh giá hiệu suất của mô hình Transformer cho dịch máy Anh-Việt.

3.  **Methods:**
    *   Sử dụng phương pháp tiếp cận dựa trên mô hình Transformer, một kiến trúc tiên tiến trong xử lý ngôn ngữ tự nhiên.
    *   Sử dụng tập dữ liệu song song PhoMT do VinAI Research cung cấp, bao gồm 3,015,